# Proyecto 3 - BD2

Importamos la libería face_recognition y probamos que efectivamente detecte si 2 rostros son iguales.

In [10]:
import face_recognition

path1 = "./vindiesel.jpg"
path2 = "./vindiesel2.jpg"
path3 = "./michelleRodriguez.jpg"

image1 = face_recognition.load_image_file(path1)
image2 = face_recognition.load_image_file(path2)
image3 = face_recognition.load_image_file(path3)

face_encoding1 = face_recognition.face_encodings(image1)[0]
face_encoding2 = face_recognition.face_encodings(image2)[0]
face_encoding3 = face_recognition.face_encodings(image3)[0]

comp1 = face_recognition.compare_faces([face_encoding2], face_encoding1)
comp2 = face_recognition.compare_faces([face_encoding3], face_encoding1)

print(comp1)
print(comp2)


[True]
[False]


Hemos confirmado que la librería funciona. Ahora, implementamos la búsqueda secuencial. Para ello, utilizaremos un maxHeap que guarde en su raíz siempre al k peor candidato. Cuando venga un mejor candidato, podremos hacer pop del k peor candidato e insertar en el heap al nuevo candidato en la posición a la que corresponda.

In [19]:
import os
import heapq        #en python no hay implementación de un maxheap, para ello multiplicaremos por -1 las distancias en el heap

def sequential_knn(k, target_face_encoding):
    dataset_path = "./lfw-chikito"
    face_encodings = []

    # Recorre las subcarpetas del dataset
    for person_folder in os.listdir(dataset_path):
        person_folder_path = os.path.join(dataset_path, person_folder)

        # Recorre las imágenes de cada persona
        for image_file in os.listdir(person_folder_path):
            image_path = os.path.join(person_folder_path, image_file)

            # Carga la imagen y extrae el código de codificación facial
            image = face_recognition.load_image_file(image_path)
            face_locations = face_recognition.face_locations(image)

            # Verifica si se detectó al menos una cara en la imagen
            if len(face_locations) > 0:
                encoding = face_recognition.face_encodings(image, face_locations)[0]

                # Calcula la distancia entre el código de codificación facial objetivo y el actual
                distance = -face_recognition.face_distance([target_face_encoding], encoding)[0]  # Invierte el signo tal que el mayor del heap siempre estará en la raíz

                # Si el heap no tiene k elementos, agrega el candidato actual
                if len(face_encodings) < k:
                    heapq.heappush(face_encodings, (distance, person_folder))
                else:
                    #
                    if abs(distance) < abs(face_encodings[0][0]):  # si la distancia del candidato es menor al mayor del heap,
                        heapq.heappushpop(face_encodings, (distance, person_folder))    #quita el mayor e inserta el candidato


    # Multiplicar por -1 a las distancias para regresarlas a la normalidad
    face_encodings = [(-x, y) for x, y in face_encodings]

    # Ordenar los resultados por distancia de manera ascendente. Es necesario porque estoy usando maxHeap
    top_k_faces = sorted(face_encodings, key=lambda x: abs(x[0]))

    return top_k_faces


# Cargar la imagen de la persona que se busca
target_image_path = "./lfw-chikito/AJ_Cook/AJ_Cook_0001.jpg"
target_image = face_recognition.load_image_file(target_image_path)
target_face_encoding = face_recognition.face_encodings(target_image)[0]

# Definir el valor de k (por ejemplo, 5)
k = 5

# Realizar la búsqueda secuencial KNN
top_k_faces = sequential_knn(k, target_face_encoding)

# Imprimir los resultados
print(f"Las {k} mejores coincidencias de rostros son: {top_k_faces}")

Las 5 mejores coincidencias de rostros son: [(0.0, 'AJ_Cook'), (0.7326286046623673, 'Agnes_Bruckner'), (0.7612283762418859, 'Aaron_Guiel'), (0.7845939997728305, 'Afton_Smith'), (0.7905119064520651, 'Aileen_Riggin_Soule')]
